Environment

In [ ]:
import os
import sys
sys.path.append("./../../../")

from ronek import env
env.set(
  device="cpu",
  device_idx=1,
  nb_threads=16,
  floatx="float64"
)

Import libraries

In [ ]:
import torch
import numpy as np

from ronek import const
from ronek.systems import TASystem
from ronek.postproc import plotting as pltt
from ronek.postproc import animation as anim
from matplotlib import pyplot as plt
from silx.io.dictdump import h5todict

Define inputs

In [ ]:
# System
T = 1e4
T0 = 4000
# Paths
paths = {
  "lev": "/home/zanardi/Codes/TC/HyperNet/database/levels/O3_UMN/O2.csv",
  "sol": f"/home/zanardi/Workspace/AirDatabase/ThermoChemModels/StS/run/O3_UMN/T0_{int(T0)}K/output_box",
  "dtb": "./../database/",
  "data": "./data/"
}
# Plotting
saving = True
showing = True

In [2]:
rom_dims = list(range(15,41,5))

[15, 20, 25, 30, 35, 40]

In [ ]:
if saving:
  os.makedirs(paths["data"]+"/figs/", exist_ok=True)

Read levels and StS solution

In [ ]:
lev, sol = read_lev_sol(
  lev_file=paths["lev"],
  path_to_sol=paths["sol"],
  molecule=molecule
)

In [ ]:
t = sol["t"]
na, nm = sol[f"X_{atom}"] * sol["n"], sol[f"n_{molecule}"]
nb_pts = len(t)

In [ ]:
# Plot distributions
frames = [0, 500, 750, 850, 1000, nb_pts-1]
for i in frames:
  ni = sol[f"n_{molecule}"][i] / lev["g"]
  it = str(i+1).zfill(4)
  plot_2D(
    x=lev['E'],
    y_true=ni,
    scales=["linear", "log"],
    filename=paths["bpod"] + f"/figs/sol_2d_i{it}.png",
    save=saving,
    show=showing
  )
  plot_3D(
    x=lev['EVib'],
    y=lev['ERot'],
    z_true=ni,
    filename=paths["bpod"] + f"/figs/sol_3d_i{it}.png",
    save=saving,
    show=showing
  )

Initialize isothermal master equation model

In [ ]:
model = TASystem(
  rates=paths["dtb"] + "/kinetics.hdf5",
  species={
    k: paths["dtb"] + f"/species/{k}.json" for k in ("atom", "molecule")
  },
  use_einsum=False
)
model.update_fom_ops(T)

In [ ]:
ei = model.species["molecule"].lev['e'] / const.eV_to_J

Balanced truncation

In [ ]:
if saving:
  os.makedirs(paths["data"]+"/figs/bases/", exist_ok=True)

In [ ]:
bases = h5todict(paths["data"]+"/bases.hdf5")
s, phi, psi = [bases[k] for k in ("s", "phi", "psi")]

In [ ]:
cs = 1.0 / np.sum(s**2)
cs *= np.cumsum(s**2)

In [ ]:
# Number of principal components
eps = 1e-5
romdim = np.where(cs > 1-eps)[0][0]+1
romdim

In [ ]:
dim_max = 20
pltt.cum_energy(
  cs[:dim_max],
  filename=paths["data"]+"/figs/bases/cum_en.png",
  save=saving,
  show=showing
)

In [ ]:
for i in range(dim_max):
  nb = str(i+1)
  pltt.dist_2d(
    x=ei,
    y_true=phi[:,i],
    labels=[r"$\epsilon_i$ [eV]", r"$\phi_{{%s}}$" % nb],
    scales=["linear", "linear"],
    filename=paths["data"] + f"/figs/bases/phi_{nb.zfill(2)}.png",
    save=saving,
    show=showing
  )
  pltt.dist_2d(
    x=ei,
    y_true=psi[:,i],
    labels=[r"$\epsilon_i$ [eV]", r"$\psi_{{%s}}$" % nb],
    scales=["linear", "linear"],
    filename=paths["bpod"] + f"/figs/bases/psi_{nb.zfill(2)}.png",
    save=saving,
    show=showing
  )

ROM Model - Testing

In [ ]:
rom_dim = 40

In [ ]:
model.set_basis(phi=phi[:,:rom_dim], psi=psi[:,:rom_dim])
model.update_rom_ops()

In [ ]:
y0 = np.concatenate([na[:1], model.psi.T @ nm[0]])
yr = model.solve(t, y0, ops=model.rom_ops, rtol=1e-7, atol=0.0)
na_pred = yr[:1]
nm_pred = model.phi @ yr[1:]

FOM vs. ROM

In [ ]:
100 * np.mean(np.abs(na_pred - na) / np.abs(na))

In [ ]:
plt.loglog(t, na)
plt.loglog(t, na_pred.squeeze(), ls='--')

In [ ]:
one = np.ones_like(lev["E"])
m = nm @ one
m_pred = nm_pred.T @ one
100 * np.mean(np.abs(m_pred - m) / np.abs(m))

In [ ]:
plt.loglog(t, m)
plt.loglog(t, m_pred, ls='--')

In [ ]:
e = nm @ lev["E"] / m
e_pred = nm_pred.T @ lev["E"] / m_pred
100 * np.mean(np.abs(e_pred - e) / np.abs(e))

In [ ]:
plt.semilogx(t, e)
plt.semilogx(t, e_pred, ls='--')

In [ ]:
# Plot distributions
for i in frames:
  plot_2D(
    x=lev['E'],
    y_true=sol[f"n_{molecule}"][i] / lev["g"],
    y_pred=nm_pred[:,i] / lev["g"],
    scales=["linear", "log"],
    filename=paths["bpod"] + f"/figs/sol_2d_i{str(i).zfill(4)}_rom.png",
    save=saving,
    show=showing
  )

In [ ]:
# g = lev["g"].reshape(1,-1)
# y = {
#   "FOM": sol[f"n_{molecule}"] / g,
#   "ROM": nm_pred.T / g
# }

In [ ]:
# animate(
#   t=t,
#   x=lev['E'],
#   y=y,
#   frames=100,
#   fps=10,
#   filename=paths["bpod"] + f"/figs/lev_dist_T0{int(T0)}K_r{rom_dim}.gif",
#   save=True,
#   show=False
# )